# Постановка задачи

По некоторому устройству по типу _Arb PUF_ были полученые пары запрос-ответ (или _Challenge-Response pairs_, _CRP_). Запрос состоит из **N** бит, где каждый бит отвечает за выбранную конфигурацию перехода сигнала: например, **0** означает, что сигналы продолжили идти параллельно каждый по своему каналу, **1** - сигналы поменялись каналами местами), а ответ содержит лишь один бит, который говорит о том, на какой из двух каналов сигнал добрался раньше. Теперь необходимо создать некий алгоритм, который умеет по предоставленному запросу, состоящему из **N** бит, давать однобитный ответ.

Tакую задачу можно рассматривать как задачу классификации. Действительно, если представить запрос как включения/исключения тех или иных характеристик (есть **N** характеристик, биты соответствуют вкл/искл), а ответ как принадлежность к определенной категории, которых всего 2. 
Если бы вместо ответа $\{0, 1\}$ давалось время отклика (или разница времен, когда сигналы дошли до конечного прибора), то эту задачу можно было бы отнести к задаче регрессии, поскольку тогда нужно было бы определить функцию $f : R^n \rightarrow R$.

Кроме того, задача является задачей *supervised*, или обучением с учителем, поскольку есть вектор-ответ *y*.

Так что исходная задача является задачей классификации, причем бинарной.

#  Возможные методы решения

Раз задача является задачей классификации, тем более бинарной, то одним из ее методов решения должна быть стандартная **логистическая регрессия**.

Кроме того, *бинарная* классификация наталкивает на мысль о возможном использовании **SVM**. Поскольку в базовом приближении SVM находит веса и байесы (разделяет гиперплоскостью), что является линейным разделением, а исходную задачу сложно назвать линейной, то необходимо выполнить т.н. *kernel trick*, то есть перейти к нелинейному разделению.

Задачу классификации также решает алгоритм **k-nearest neighbours**. Но сразу хочется заметить, что здесь сложно дать понятие близости запросов, поскольку если порассуждать над значением N-битового входа, то процесс распространения сигнала становится очень запутанным даже при одном-двух изменениях в битах между парой запросов. С другой стороны, если правильно интерпретировать входные данные (сделать некое преобразование над входом), чтобы такая близость была более очевидной, то этот алгоритм должен показать хорошие результаты.

**Дерево решений** - еще один алгоритм для рассмотрения. Но здесь, как и в случае с *k-nearest neighbours*, однозначно стоит сделать преобразование над входом, поскольку интерпретировать N-битное число как натуральное число является априори неверным подходом. Поэтому здесь трудность будет найти подходящую функцию преобразования.

Ну и на закуску можно предложить поиграться с нейронными сетями...

# Обучающая выборка

Сразу стоит отметить, что мощность полезной обучающей выборки не может превысить $2^N$, следуя простым соображениям. Но понятно, что универсальное множество данных для N обучать не стоит, поскольку имея такую таблицу никакую задачу решать и не стоит.

Оценка снизу должна состоять из следующих соображений. Во-первых, для каждой позиции бита среди данных должен найтись пара различных запросов с разными значениями бита в них. Если пойти дальше, то можно предположить, что, рассмотрев все возможные упорядоченные $k$-множества индексов (индекс $\in \{1..N\}$) для некоторого $k$, можно построить хорошую взаимосвязь между блоками переключения и понять, как они влияют на сигнал (замедляют/ускоряют). Размер входа в этом случае должен быть порядка $T = C_N^k * 2^k$, что для $N=80$, $k=2$ равно $T = C_{80}^2 * 2^2 = 12640$. Но это с учетом того, что все данные в выборке имеют только $k$ единиц на каждый запрос. С шумом можно представлять необходимое количество входные данных как $O(N^2)$